In [2]:
import polars as pl
import matplotlib.pyplot as plt


FILE_PATH = "../csv_files/fma_metadata/tracks.csv"

In [3]:
df = pl.read_csv(FILE_PATH, has_header=False)
df

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49,column_50,column_51,column_52,column_53
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
null,"""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""album""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""artist""","""set""","""set""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track""","""track"""
null,"""comments""","""date_created""","""date_released""","""engineer""","""favorites""","""id""","""information""","""listens""","""producer""","""tags""","""title""","""tracks""","""type""","""active_year_begin""","""active_year_end""","""associated_labels""","""bio""","""comments""","""date_created""","""favorites""","""id""","""latitude""","""location""","""longitude""","""members""","""name""","""related_projects""","""tags""","""website""","""wikipedia_page""","""split""","""subset""","""bit_rate""","""comments""","""composer""","""date_created""","""date_recorded""","""duration""","""favorites""","""genre_top""","""genres""","""genres_all""","""information""","""interest""","""language_code""","""license""","""listens""","""lyricist""","""number""","""publisher""","""tags""","""title"""
"""track_id""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""small""","""256000""","""0""",null,"""2008-11-26 01:48:12""","""2008-11-26 00:00:00""","""168""","""2""","""Hip-Hop""","""[21]""","""[21]""",null,"""4656""","""en""","""Attribution-NonCommercial-Shar…","""1293""",null,"""3""",null,"""[]""","""Food"""
"""3""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""medium""","""256000""","""0""",null,"""2008-11-26 01:48:14""","""2008-11-26 00:00:00""","""237""","""1""","""Hip-Hop""","""[21]""","""[21]""",null,"""1470""","""en""","""Attribution-NonCommercial-Shar…","""514""",null,"""4""",null,"""[]""","""Electric Ave"""
…,…,…,…,…,…,…,…,…,…,…,

In [4]:
# Grab the first two “header” rows
header = list(df.row(1))     # e.g. [None, "comments", "date_created", …]
header[0] = "track_id"

def make_unique_columns(cols):
    seen = {}
    result = []
    for col in cols:
        if col in seen:
            seen[col] += 1
            new_col = f"{col}_{seen[col]}"
        else:
            seen[col] = 0
            new_col = col
        result.append(new_col)
    return result

unique_header = make_unique_columns(header)

# Drop those two rows of “headers” from the data
new_df = df.slice(3)

# Rename all columns in one go
new_df = new_df.rename({old: new for old, new in zip(df.columns, unique_header)})


new_df

track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments_1,date_created_1,favorites_1,id_1,latitude,location,longitude,members,name,related_projects,tags_1,website,wikipedia_page,split,subset,bit_rate,comments_2,composer,date_created_2,date_recorded,duration,favorites_2,genre_top,genres,genres_all,information_1,interest,language_code,license,listens_1,lyricist,number,publisher,tags_2,title_1
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""small""","""256000""","""0""",null,"""2008-11-26 01:48:12""","""2008-11-26 00:00:00""","""168""","""2""","""Hip-Hop""","""[21]""","""[21]""",null,"""4656""","""en""","""Attribution-NonCommercial-Shar…","""1293""",null,"""3""",null,"""[]""","""Food"""
"""3""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""medium""","""256000""","""0""",null,"""2008-11-26 01:48:14""","""2008-11-26 00:00:00""","""237""","""1""","""Hip-Hop""","""[21]""","""[21]""",null,"""1470""","""en""","""Attribution-NonCommercial-Shar…","""514""",null,"""4""",null,"""[]""","""Electric Ave"""
"""5""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""small""","""256000""","""0""",null,"""2008-11-26 01:48:20""","""2008-11-26 00:00:00""","""206""","""6""","""Hip-Hop""","""[21]""","""[21]""",null,"""1933""","""en""","""Attribution-NonCommercial-Shar…","""1151""",null,"""6""",null,"""[]""","""This World"""
"""10""","""0""","""2008-11-26 01:45:08""","""2008-02-06 00:00:00""",null,"""4""","""6""",null,"""47632""",null,"""[]""","""Constant Hitmaker""","""2""","""Album""",null,null,"""Mexican Summer, Richie Records…","""<p><span style=""font-family:Ve…","""3""","""2008-11-26 01:42:55""","""74""","""6""",null,null,null,"""Kurt Vile, the Violators""","""Kurt Vile""",null,"""['philly', 'kurt vile']""","""http://kurtvile.com""",null,"""training""","""small""","""192000""","""0""","""Kurt Vile""","""2008-11-25 17:49:06""","""2008-11-26 00:00:00""","""161""","""178""","""Pop""","""[10]""","""[10]""",null,"""54881""","""en""","""Attribution-NonCommercial-NoDe…","""50135""",null,"""1""",null,"""[]""","""Freeway"""
"""20""","""0""","""2008-11-26 01:45:05""","""2009-01-06 00:00:00""",null,"""2""","""4""","""<p> ""spiritual songs"" from Nic…","""2710""",null,"""[]""","""Niris""","""13""","""Album""","""1990-01-01 00:00:00""","""2011-01-01 00:00:00""",null,"""<p>Songs written by: Nick

In [5]:
df_small = new_df.filter(pl.col("subset") == "small")
df_small

track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments_1,date_created_1,favorites_1,id_1,latitude,location,longitude,members,name,related_projects,tags_1,website,wikipedia_page,split,subset,bit_rate,comments_2,composer,date_created_2,date_recorded,duration,favorites_2,genre_top,genres,genres_all,information_1,interest,language_code,license,listens_1,lyricist,number,publisher,tags_2,title_1
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""small""","""256000""","""0""",null,"""2008-11-26 01:48:12""","""2008-11-26 00:00:00""","""168""","""2""","""Hip-Hop""","""[21]""","""[21]""",null,"""4656""","""en""","""Attribution-NonCommercial-Shar…","""1293""",null,"""3""",null,"""[]""","""Food"""
"""5""","""0""","""2008-11-26 01:44:45""","""2009-01-05 00:00:00""",null,"""4""","""1""","""<p></p>""","""6073""",null,"""[]""","""AWOL - A Way Of Life""","""7""","""Album""","""2006-01-01 00:00:00""",null,null,"""<p>A Way Of Life, A Collective…","""0""","""2008-11-26 01:42:32""","""9""","""1""","""40.0583238""","""New Jersey""","""-74.4056612""","""Sajje Morocco,Brownbum,ZawidaG…","""AWOL""","""The list of past projects is 2…","""['awol']""","""http://www.AzillionRecords.blo…",null,"""training""","""small""","""256000""","""0""",null,"""2008-11-26 01:48:20""","""2008-11-26 00:00:00""","""206""","""6""","""Hip-Hop""","""[21]""","""[21]""",null,"""1933""","""en""","""Attribution-NonCommercial-Shar…","""1151""",null,"""6""",null,"""[]""","""This World"""
"""10""","""0""","""2008-11-26 01:45:08""","""2008-02-06 00:00:00""",null,"""4""","""6""",null,"""47632""",null,"""[]""","""Constant Hitmaker""","""2""","""Album""",null,null,"""Mexican Summer, Richie Records…","""<p><span style=""font-family:Ve…","""3""","""2008-11-26 01:42:55""","""74""","""6""",null,null,null,"""Kurt Vile, the Violators""","""Kurt Vile""",null,"""['philly', 'kurt vile']""","""http://kurtvile.com""",null,"""training""","""small""","""192000""","""0""","""Kurt Vile""","""2008-11-25 17:49:06""","""2008-11-26 00:00:00""","""161""","""178""","""Pop""","""[10]""","""[10]""",null,"""54881""","""en""","""Attribution-NonCommercial-NoDe…","""50135""",null,"""1""",null,"""[]""","""Freeway"""
"""140""","""1""","""2008-11-26 01:49:59""","""2007-05-22 00:00:00""",null,"""1""","""61""","""<p>Alec K. Redfearn &amp; The …","""1300""","""Alec K. Refearn, Rob Pemberton""","""[]""","""The Blind Spot""","""1""","""Album""","""1999-01-01 00:00:00""",null,null,"""<p>The Eyesores originally for…","""0""","""2008-11-26 01:47:44""","""11""","""54""","""41.8239891""","""Providence, RI""","""-71.4128343""",null,"""Alec K. Redfearn & the Eyesore…","""Haldols, Amoebic Ensemble, Mr.…","""['alec k redfearn and the eyes…","""http://www.aleckredfearn.com""","""http://en.wikipedia.org/wiki/A…","""training""","""small""","""128000""","""0""",null,"""2008-11-26 01:44:07""","""2008-11-26 00:00:00""","""253""","""5""","""Folk""","""[17]""","""[17]""",null,"""1593""","""en""","""Attribution-Noncommercial-No D…","""1299""",null,"""2""",null,"""[]""","""Queen Of The Wires"""
"""141""","""0""","""2008-11-26 01:49:57""","""2009-01-16 00:00:00""",null,"""1""","""60""","""<p>A full ensamble of strings,…","""1304""",null,"""[]""","""Every Man 

In [6]:
new_df.write_csv("./processed_small_dataset.csv", separator=",")